# Практическая работа

# Задача

Один из способов повысить эффективность взаимодействия банка с клиентами — отправлять предложение о новой услуге не всем клиентам, а только некоторым, которые выбираются по принципу наибольшей склонности к отклику на это предложение.

Задача заключается в том, чтобы предложить алгоритм, который будет выдавать склонность клиента к положительному или отрицательному отклику на предложение банка. Предполагается, что, получив такие оценки для некоторого множества клиентов, банк обратится с предложением только к тем, от кого ожидается положительный отклик.


Для решения этой задачи загрузите файлы из базы в Postgres.
Эта БД хранит информацию о клиентах банка и их персональные данные, такие как пол, количество детей и другие.

Описание таблиц с данными представлено ниже.


**D_work**

Описание статусов относительно работы:
- ID — идентификатор социального статуса клиента относительно работы;
- COMMENT — расшифровка статуса.


**D_pens**

Описание статусов относительно пенсии:
- ID — идентификатор социального статуса;
- COMMENT — расшифровка статуса.


**D_clients**

Описание данных клиентов:
- ID — идентификатор записи;
- AGE	— возраст клиента;
- GENDER — пол клиента (1 — мужчина, 0 — женщина);
- EDUCATION — образование;
- MARITAL_STATUS — семейное положение;
- CHILD_TOTAL	— количество детей клиента;
- DEPENDANTS — количество иждивенцев клиента;
- SOCSTATUS_WORK_FL	— социальный статус клиента относительно работы (1 — работает, 0 — не работает);
- SOCSTATUS_PENS_FL	— социальный статус клиента относительно пенсии (1 — пенсионер, 0 — не пенсионер);
- REG_ADDRESS_PROVINCE — область регистрации клиента;
- FACT_ADDRESS_PROVINCE — область фактического пребывания клиента;
- POSTAL_ADDRESS_PROVINCE — почтовый адрес области;
- FL_PRESENCE_FL — наличие в собственности квартиры (1 — есть, 0 — нет);
- OWN_AUTO — количество автомобилей в собственности.


**D_agreement**

Таблица с зафиксированными откликами клиентов на предложения банка:
- AGREEMENT_RK — уникальный идентификатор объекта в выборке;
- ID_CLIENT — идентификатор клиента;
- TARGET — целевая переменная: отклик на маркетинговую кампанию (1 — отклик был зарегистрирован, 0 — отклика не было).
    
    
**D_job**

Описание информации о работе клиентов:
- GEN_INDUSTRY — отрасль работы клиента;
- GEN_TITLE — должность;
- JOB_DIR — направление деятельности внутри компании;
- WORK_TIME — время работы на текущем месте (в месяцах);
- ID_CLIENT — идентификатор клиента.


**D_salary**

Описание информации о заработной плате клиентов:
- ID_CLIENT — идентификатор клиента;
- FAMILY_INCOME — семейный доход (несколько категорий);
- PERSONAL_INCOME — личный доход клиента (в рублях).


**D_last_credit**

Информация о последнем займе клиента:
- ID_CLIENT — идентификатор клиента;
- CREDIT — сумма последнего кредита клиента (в рублях);
- TERM — срок кредита;
- FST_PAYMENT — первоначальный взнос (в рублях).


**D_loan**

Информация о кредитной истории клиента:
- ID_CLIENT — идентификатор клиента;
- ID_LOAN — идентификатор кредита.

**D_close_loan**

Информация о статусах кредита (ссуд):
- ID_LOAN — идентификатор кредита;
- CLOSED_FL — текущий статус кредита (1 — закрыт, 0 — не закрыт).

Ниже представлен минимальный список колонок, которые должны находиться в итоговом датасете после склейки и агрегации данных. По своему усмотрению вы можете добавить дополнительные к этим колонки.

    - AGREEMENT_RK — уникальный идентификатор объекта в выборке;
    - TARGET — целевая переменная: отклик на маркетинговую кампанию (1 — отклик был зарегистрирован, 0 — отклика не было);
    - AGE — возраст клиента;
    - SOCSTATUS_WORK_FL — социальный статус клиента относительно работы (1 — работает, 0 — не работает);
    - SOCSTATUS_PENS_FL — социальный статус клиента относительно пенсии (1 — пенсионер, 0 — не пенсионер);
    - GENDER — пол клиента (1 — мужчина, 0 — женщина);
    - CHILD_TOTAL — количество детей клиента;
    - DEPENDANTS — количество иждивенцев клиента;
    - PERSONAL_INCOME — личный доход клиента (в рублях);
    - LOAN_NUM_TOTAL — количество ссуд клиента;
    - LOAN_NUM_CLOSED — количество погашенных ссуд клиента.


Будьте внимательны при сборке датасета: это реальные банковские данные, в которых могут наблюдаться дубли, некорректно заполненные значения или значения, противоречащие друг другу. Для получения качественной модели необходимо предварительно очистить датасет от такой информации.

## Задание 1

Соберите всю информацию о клиентах в одну таблицу, где одна строчка соответствует полной информации об одном клиенте.

In [1]:
import numpy as np
import pandas as pd

#### Загружаем таблицы

In [2]:
df_clients = pd.read_csv('./datasets/D_clients.csv')
df_target = pd.read_csv('./datasets/D_target.csv')
df_salary = pd.read_csv('./datasets/D_salary.csv')
df_loan = pd.read_csv('./datasets/D_loan.csv')
df_close_loan = pd.read_csv('./datasets/D_close_loan.csv')

#### Считаем LOAN_NUM_TOTAL и LOAN_NUM_CLOSED

In [3]:
df_loan_all = df_loan.merge(df_close_loan, how = 'left', left_on = 'ID_LOAN', right_on = 'ID_LOAN').drop(['ID_LOAN'], axis=1)
df_loan_all = df_loan_all.groupby('ID_CLIENT').agg(['count', 'sum'])
df_loan_all.columns = ['LOAN_NUM_TOTAL', 'LOAN_NUM_CLOSED']
df_loan_all.reset_index(inplace=True)

#### Присоединяем таблицы

In [4]:
df = df_clients.merge(df_target, how = 'inner', left_on = 'ID', right_on = 'ID_CLIENT').drop(['ID_CLIENT'], axis=1)

In [5]:
df = df.merge(df_salary, how = 'left', left_on = 'ID', right_on = 'ID_CLIENT')\
    .drop(['ID_CLIENT', 'FAMILY_INCOME'], axis=1)

In [6]:
df = df.merge(df_loan_all, how = 'left', left_on = 'ID', right_on = 'ID_CLIENT').drop(['ID_CLIENT'], axis=1)

In [7]:
cols = ['AGREEMENT_RK',
'TARGET',
'AGE',
'SOCSTATUS_WORK_FL',
'SOCSTATUS_PENS_FL',
'GENDER',
'CHILD_TOTAL',
'DEPENDANTS',
'PERSONAL_INCOME',
'LOAN_NUM_TOTAL',
'LOAN_NUM_CLOSED']

In [8]:
df = df[cols]

In [9]:
df.reset_index(inplace=True, drop=True)

#### Пропуски

In [10]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 15523 entries, 0 to 15522
Data columns (total 11 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   AGREEMENT_RK       15523 non-null  int64  
 1   TARGET             15523 non-null  int64  
 2   AGE                15523 non-null  int64  
 3   SOCSTATUS_WORK_FL  15523 non-null  int64  
 4   SOCSTATUS_PENS_FL  15523 non-null  int64  
 5   GENDER             15523 non-null  int64  
 6   CHILD_TOTAL        15523 non-null  int64  
 7   DEPENDANTS         15523 non-null  int64  
 8   PERSONAL_INCOME    15523 non-null  float64
 9   LOAN_NUM_TOTAL     15523 non-null  int64  
 10  LOAN_NUM_CLOSED    15523 non-null  int64  
dtypes: float64(1), int64(10)
memory usage: 1.3 MB


#### Статистики

In [11]:
df.describe(include='all')

,AGREEMENT_RK,TARGET,AGE,SOCSTATUS_WORK_FL,SOCSTATUS_PENS_FL,GENDER,CHILD_TOTAL,DEPENDANTS,PERSONAL_INCOME,LOAN_NUM_TOTAL,LOAN_NUM_CLOSED
count,1.552300e+04,15523.000000,15523.000000,15523.000000,15523.000000,15523.000000,15523.000000,15523.000000,15523.000000,15523.000000,15523.000000
mean,6.539832e+07,0.119371,40.400438,0.909360,0.134639,0.653997,1.098886,0.644850,13848.041638,1.387296,0.751594
std,4.569483e+06,0.324235,11.607242,0.287105,0.341348,0.475709,0.996748,0.812663,8998.618992,0.794241,0.989253
min,5.991015e+07,0.000000,21.000000,0.000000,0.000000,0.000000,0.000000,0.000000,24.000000,1.000000,0.000000
25%,6.191984e+07,0.000000,30.000000,1.000000,0.000000,0.000000,0.000000,0.000000,8000.000000,1.000000,0.000000
50%,6.436952e+07,0.000000,39.000000,1.000000,0.000000,1.000000,1.000000,0.000000,12000.000000,1.000000,0.000000
75%,6.708832e+07,0.000000,50.000000,1.000000,0.000000,1.000000,2.000000,1.000000,17000.000000,2.000000,1.000000
max,7.529224e+07,1.000000,67.000000,1.000000,1.000000,1.000000,10.000000,7.000000,250000.000000,11.000000,11.000000


In [12]:
df.duplicated().any()

True

#### Найдены выбросы в PERSONAL_INCOME

In [13]:
df[df['PERSONAL_INCOME'] < 2000]

,AGREEMENT_RK,TARGET,AGE,SOCSTATUS_WORK_FL,SOCSTATUS_PENS_FL,GENDER,CHILD_TOTAL,DEPENDANTS,PERSONAL_INCOME,LOAN_NUM_TOTAL,LOAN_NUM_CLOSED
3272,61534026,0,28,1,0,0,1,1,24.0,1,0
14354,65421532,0,23,0,1,1,0,0,1950.0,1,1


In [14]:
df = df[df['PERSONAL_INCOME'] >= 1950].reset_index(drop=True)

In [15]:
df.to_csv('clients.csv')

## Задание 2

При помощи инструмента Streamlit проведите разведочный анализ данных. В него может входить:

* построение графиков распределений признаков
* построение матрицы корреляций
* построение графиков зависимостей целевой переменной и признаков
* вычисление числовых характеристик распределения числовых столбцов (среднее, min, max, медиана и так далее)
* любые другие ваши идеи приветствуются!

[Пример Streamlit-приложения](https://rateyourflight.streamlit.app) с разведочным анализом, прогнозом модели и оценкой ее результатов.